Importing libraries 

In [ ]:
#importing relevant libraries
import pandas as pd
from collections import OrderedDict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import math
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

Loading and prepocessing the data

In [ ]:
# load the mock data into a pandas dataframe

courses_df = pd.read_csv('course_catalogue.csv')
students_courses_df = pd.read_csv('historical_data.csv')

In [ ]:
# Display the first few rows of the dataframes
courses_df.head()

,Course_name,Description,Keywords,Core_units,Reviews,Weighted_points,Job_satisfaction
0,Medicine and Surgery,This course trains students to become medical ...,"Medicine, Surgery, Health","Anatomy, Physiology, Pharmacology, Pathology",4.7,43.68,5.0
1,Pharmacy,This course trains students to become licensed...,"Pharmacy, Medications, Health","Pharmacology, Medicinal Chemistry, Pharmacy Pr...",4.2,42.10,4.5
2,Nursing,This course trains students to become register...,"Anatomy, Physiology, Medical-Surgical Nursing","Anatomy, Physiology, Pharmacology, Pathology",4.5,39.47,4.0
3,Medical Laboratory Sciences,This course trains students to become medical ...,"Medical Laboratory, Diagnostics, Health","Medical Microbiology, Hematology, Clinical Che...",4.5,37.89,4.5
4,Nutrition and Dietetics,This course trains students to become register...,"Nutrition, Dietetics, Health","Human Nutrition, Medical Nutrition Therapy, Fo...",4.3,36.84,4.0


In [ ]:
students_courses_df.head()

,ID,First_Name,Last_Name,English,Kiswahili,Mathematics,Physics,Biology,Chemistry,Overall Grade,Interest,Cluster Points,Course
0,S001,Scarlett,Layla,10,9,10,7,6,10,61,Public Health,35.424,Health Records and Information Technology
1,S002,Amelia,Zoey,12,11,6,7,12,7,67,Pharmacy,38.143,Not elligible
2,S003,Emma,Aubrey,7,8,7,8,12,12,81,Laboratories,43.028,Medical Laboratory Sciences
3,S004,Luna,Cooper,11,8,7,6,11,12,63,Nursing,38.419,Palliative Care and Hospice Services
4,S005,Jackson,Adams,7,6,10,10,10,9,64,Surgery,36.285,Not elligible


In [ ]:
students_courses_df.shape

(1000, 13)

In [ ]:
# make copies of the dataframes
courses_copy = courses_df.copy()


In [ ]:
def lang(students):
  if students['English'] >= students['Kiswahili']:
    return students['English']
  else:
    return students['Kiswahili']
    
def science(students):
  if students['Mathematics'] >= students['Physics']:
    return students['Mathematics']
  else:
    return students['Physics']

In [ ]:
students_courses_df['languages'] = students_courses_df.apply(lang, axis=1)
students_courses_df['science'] = students_courses_df.apply(science, axis=1)
students_courses_df.head()

,ID,First_Name,Last_Name,English,Kiswahili,Mathematics,Physics,Biology,Chemistry,Overall Grade,Interest,Cluster Points,Course,languages,science
0,S001,Scarlett,Layla,10,9,10,7,6,10,61,Public Health,35.424,Health Records and Information Technology,10,10
1,S002,Amelia,Zoey,12,11,6,7,12,7,67,Pharmacy,38.143,Not elligible,12,7
2,S003,Emma,Aubrey,7,8,7,8,12,12,81,Laboratories,43.028,Medical Laboratory Sciences,8,8
3,S004,Luna,Cooper,11,8,7,6,11,12,63,Nursing,38.419,Palliative Care and Hospice Services,11,7
4,S005,Jackson,Adams,7,6,10,10,10,9,64,Surgery,36.285,Not elligible,7,10


In [ ]:
students_courses_df.columns

Index(['ID', 'First_Name', 'Last_Name', 'English', 'Kiswahili', 'Mathematics',
       'Physics', 'Biology', 'Chemistry', 'Overall Grade', 'Interest',
       'Cluster Points', 'Course', 'languages', 'science'],
      dtype='object')

In [ ]:
students_course = students_courses_df.drop(students_courses_df[students_courses_df['Course'] == 'Not elligible'].index).reset_index(drop=True)
students_course.sample(5)

,ID,First_Name,Last_Name,English,Kiswahili,Mathematics,Physics,Biology,Chemistry,Overall Grade,Interest,Cluster Points,Course,languages,science
488,S725,Stella,Claire,6,9,9,10,7,8,63,Nursing,34.986,Palliative Care and Hospice Services,9,10
17,S027,Moore,Stella,9,8,6,9,8,11,71,Therapy,38.745,Orthotics and Prosthetics,9,9
162,S236,Stella,Jones,12,11,10,8,9,6,67,Nursing,37.637,Palliative Care and Hospice Services,12,10
30,S045,Jones,Campbell,11,10,6,9,9,7,67,Medical Research,37.125,Epidemiology and Biostatistics,11,9
536,S785,Campbell,Zoey,7,9,11,10,10,11,82,Nursing,43.831,Nursing,9,11


In [ ]:
courses_copy.head()

,Course_name,Description,Keywords,Core_units,Reviews,Weighted_points,Job_satisfaction
0,Medicine and Surgery,This course trains students to become medical ...,"Medicine, Surgery, Health","Anatomy, Physiology, Pharmacology, Pathology",4.7,43.68,5.0
1,Pharmacy,This course trains students to become licensed...,"Pharmacy, Medications, Health","Pharmacology, Medicinal Chemistry, Pharmacy Pr...",4.2,42.10,4.5
2,Nursing,This course trains students to become register...,"Anatomy, Physiology, Medical-Surgical Nursing","Anatomy, Physiology, Pharmacology, Pathology",4.5,39.47,4.0
3,Medical Laboratory Sciences,This course trains students to become medical ...,"Medical Laboratory, Diagnostics, Health","Medical Microbiology, Hematology, Clinical Che...",4.5,37.89,4.5
4,Nutrition and Dietetics,This course trains students to become register...,"Nutrition, Dietetics, Health","Human Nutrition, Medical Nutrition Therapy, Fo...",4.3,36.84,4.0


In [ ]:
courses_copy.columns

Index(['Course_name', 'Description', 'Keywords', 'Core_units', 'Reviews',
       'Weighted_points', 'Job_satisfaction'],
      dtype='object')

In [ ]:
# Merge the datasets on the 'Course' column
students_courses_merged = pd.merge(students_course, courses_copy[['Course_name', 'Weighted_points', 'Job_satisfaction',  'Description', 'Keywords' ]], left_on='Course', right_on='Course_name', how='left')

# Drop the extra 'Course_name' column
students_courses_merged = students_courses_merged.drop(['Course_name'], axis=1)

# Print the merged dataset
students_courses_merged.head(10)

,ID,First_Name,Last_Name,English,Kiswahili,Mathematics,Physics,Biology,Chemistry,Overall Grade,Interest,Cluster Points,Course,languages,science,Weighted_points,Job_satisfaction,Description,Keywords
0,S001,Scarlett,Layla,10,9,10,7,6,10,61,Public Health,35.424,Health Records and Information Technology,10,10,33.89,4.5,This course focuses on the management of patie...,"health records, healthcare technology"
1,S003,Emma,Aubrey,7,8,7,8,12,12,81,Laboratories,43.028,Medical Laboratory Sciences,8,8,37.89,4.5,This course trains students to become medical ...,"Medical Laboratory, Diagnostics, Health"
2,S004,Luna,Cooper,11,8,7,6,11,12,63,Nursing,38.419,Palliative Care and Hospice Services,11,7,34.21,4.6,This course focuses on the care of patients wi...,"palliative care, hospice services, serious ill..."
3,S007,Allen,Walker,12,10,9,11,9,11,62,Medical Research,39.031,Clinical Research,12,11,36.58,4.9,This course covers the principles and practice...,"clinical research, study design, data analysis..."
4,S008,Moore,Hazel,10,6,10,9,11,10,73,Nursing,41.356,Palliative Care and Hospice Services,10,10,34.21,4.6,This course focuses on the care of patients wi...,"palliative care, hospice services, serious ill..."
5,S010,Amelia,Lily,12,8,9,12,12,10,74,Laboratories,44.104,Medical Illustration and Animation,12,12,35.00,4.8,This course covers the use of illustration and...,"medical illustration, medical animation, scien..."
6,S011,Violet,Ava,9,10,12,12,11,11,73,Therapy,42.842,Clinical Psychology,10,12,39.37,4.9,This course focuses on the diagnosis and treat...,"mental health, psychology"
7,S012,Mitchell,Savannah,9,11,6,7,11,9,60,Nursing,36.095,Palliative Care and Hospice Services,11,7,34.21,4.6,This course focuses on the care of patients wi...,"palliative care, hospice services, serious ill..."
8,S013,Rodriguez,Simpson,9,10,8,12,8,7,82,Laboratories,41.638,Medical Illustration and Animation,10,12,35.00,4.8,This course covers the use of illustration and...,"medical illustration, medical animation, scien..."
9,S014,Brooklyn,Smith,12,6,9,6,11,7,57,Laboratories,35.641,Medical Illustration and Animation,12,9,35.00,4.8,This course covers the use of illustration and...,"medical illustration, medical animation, scien..."


One-hot encoding of categorical variables

In [ ]:
# Convert categorical variables in the merged dataset using one-hot encoding
students_courses_merged_encoded = pd.get_dummies(students_courses_merged, columns=['Interest'])

students_courses_merged_encoded.head(10)

,ID,First_Name,Last_Name,English,Kiswahili,Mathematics,Physics,Biology,Chemistry,Overall Grade,...,Job_satisfaction,Description,Keywords,Interest_Laboratories,Interest_Medical Research,Interest_Nursing,Interest_Pharmacy,Interest_Public Health,Interest_Surgery,Interest_Therapy
0,S001,Scarlett,Layla,10,9,10,7,6,10,61,...,4.5,This course focuses on the management of patie...,"health records, healthcare technology",0,0,0,0,1,0,0
1,S003,Emma,Aubrey,7,8,7,8,12,12,81,...,4.5,This course trains students to become medical ...,"Medical Laboratory, Diagnostics, Health",1,0,0,0,0,0,0
2,S004,Luna,Cooper,11,8,7,6,11,12,63,...,4.6,This course focuses on the care of patients wi...,"palliative care, hospice services, serious ill...",0,0,1,0,0,0,0
3,S007,Allen,Walker,12,10,9,11,9,11,62,...,4.9,This course covers the principles and practice...,"clinical research, study design, data analysis...",0,1,0,0,0,0,0
4,S008,Moore,Hazel,10,6,10,9,11,10,73,...,4.6,This course focuses on the care of patients wi...,"palliative care, hospice services, serious ill...",0,0,1,0,0,0,0
5,S010,Amelia,Lily,12,8,9,12,12,10,74,...,4.8,This course covers the use of illustration and...,"medical illustration, medical animation, scien...",1,0,0,0,0,0,0
6,S011,Violet,Ava,9,10,12,12,11,11,73,...,4.9,This course focuses on the diagnosis and treat...,"mental health, psychology",0,0,0,0,0,0,1
7,S012,Mitchell,Savannah,9,11,6,7,11,9,60,...,4.6,This course focuses on the care of patients wi...,"palliative care, hospice services, serious ill...",0,0,1,0,0,0,0
8,S013,Rodriguez,Simpson,9,10,8,12,8,7,82,...,4.8,This course covers the use of illustration and...,"medical illustration, medical animation, scien...",1,0,0,0,0,0,0
9,S014,Brooklyn,Smith,12,6,9,6,11,7,57,...,4.8,This course covers the use of illustration and...,"medical illustration, medical animation, scien...",1,0,0,0,0,0,0


In [ ]:
df = students_courses_merged_encoded
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 674 entries, 0 to 673
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         674 non-null    object 
 1   First_Name                 674 non-null    object 
 2   Last_Name                  674 non-null    object 
 3   English                    674 non-null    int64  
 4   Kiswahili                  674 non-null    int64  
 5   Mathematics                674 non-null    int64  
 6   Physics                    674 non-null    int64  
 7   Biology                    674 non-null    int64  
 8   Chemistry                  674 non-null    int64  
 9   Overall Grade              674 non-null    int64  
 10  Cluster Points             674 non-null    float64
 11  Course                     674 non-null    object 
 12  languages                  674 non-null    int64  
 13  science                    674 non-null    int64  

Performing the test-train split

In [ ]:
from sklearn.model_selection import train_test_split
# Define the filter condition based on Cluster Points and Weighted_points
filter_condition = (df['Weighted_points'] <= df['Cluster Points'])

# Filter the dataset
filtered_df = df[filter_condition]# Define features and target
cols = ['ID', 'First_Name', 'Last_Name', 'Description', 'Keywords', 'Course']
X = filtered_df.drop(cols, axis=1)
y = filtered_df['Course'].to_frame()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y.columns

Index(['Course'], dtype='object')

### Random Forest Classifier

This algorithm is suitable for predicting classes from data, and it can also rank the importance of each feature in the prediction process.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# create a random forest classifier model
rfc = RandomForestClassifier()

# train the model on the training data
rfc.fit(X_train, y_train.values.ravel())

RandomForestClassifier()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# make predictions on the test data
y_pred = rfc.predict(X_test)



Evaluating model performance

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

                                           precision    recall  f1-score   support

                      Clinical Psychology       1.00      1.00      1.00         3
                        Clinical Research       1.00      1.00      1.00         7
                           Dental Surgery       0.67      1.00      0.80         2
                     Environmental Health       1.00      1.00      1.00         3
           Epidemiology and Biostatistics       0.88      1.00      0.93         7
                         Forensic Science       1.00      1.00      1.00         5
           Health Education and Promotion       1.00      1.00      1.00         3
Health Records and Information Technology       1.00      1.00      1.00         4
                Health Systems Management       1.00      1.00      1.00         2
                     Medical Anthropology       1.00      1.00      1.00         3
                    Medical Biotechnology       1.00      1.00      1.00         4
   

Conducting hyperparameter tuning to improve model performance in some classes;

In [ ]:
# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a grid search object
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='f1_macro')

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

# Make predictions on the test data using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print classification report for the best model
print(classification_report(y_test, y_pred))


Best hyperparameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
                                           precision    recall  f1-score   support

                      Clinical Psychology       0.60      1.00      0.75         3
                        Clinical Research       1.00      1.00      1.00         7
                           Dental Surgery       0.67      1.00      0.80         2
                     Environmental Health       1.00      1.00      1.00         3
           Epidemiology and Biostatistics       0.88      1.00      0.93         7
                         Forensic Science       1.00      1.00      1.00         5
           Health Education and Promotion       1.00      1.00      1.00         3
Health Records and Information Technology       1.00      1.00      1.00         4
                Health Systems Management       0.67      1.00      0.80         2
                     Medical Anthropology       1.00      1.00

Ranking and getting the courses based on probability

In [ ]:
# get the probability of each class for the test data
probs = best_model.predict_proba(X_test)

# Get the list of courses
courses = y['Course'].unique()

In [ ]:
# Get the indices of the columns representing the interests
interests_cols = [X.columns.get_loc('Interest_Laboratories'),
                  X.columns.get_loc('Interest_Medical Research'),
                  X.columns.get_loc('Interest_Nursing'),
                  X.columns.get_loc('Interest_Pharmacy'),
                  X.columns.get_loc('Interest_Public Health'),
                  X.columns.get_loc('Interest_Surgery'),
                  X.columns.get_loc('Interest_Therapy')]

# Get the interests of the test data
interests = X_test.iloc[:, interests_cols]

# Select the top 3 courses for each student based on the probabilities
top_courses = []
for i in range(probs.shape[0]):
    sorted_probs = sorted(list(enumerate(probs[i])), key=lambda x: -x[1])
    top_courses.extend([(courses[j], prob) for j, prob in sorted_probs[:3]])


In [ ]:
# Sort the recommended courses by probability
top_courses.sort(key=lambda x: -x[1])

# Remove duplicates while preserving order
from collections import OrderedDict
unique_top_courses = list(OrderedDict.fromkeys(top_courses))

# Recommend the top 3 unique courses
recommended_courses = unique_top_courses[:3]

print("Recommended courses:")
for i, course_ in enumerate(recommended_courses):
    print(f"{i+1}. {course_[0]}")

Recommended courses:
1. Occupational Therapy
2. Optometry
3. Health Education and Promotion


> Compressing the model using pickle

In [ ]:
import pickle
import gzip


# Save the trained model using pickle
with gzip.open('rfc_pickled_model.pklz', 'wb') as f:
    pickle.dump(rfc, f)

